In [1]:
# hard negative 만들기
from utils.tools import *
from utils.utils import load_jsonl
from tqdm import tqdm
from transformers import AutoTokenizer
import argparse
from retrieval.sparse_retrieval.bm25_retrieval import BM25

In [2]:
parser = argparse.ArgumentParser()
# test name
parser.add_argument('--data_path', type=str, default='./data/wiki.json')
parser.add_argument('--test_data_path', type=str, default='./data/train.jsonl')
parser.add_argument('--include_title', type=str2bool, default=True)

_StoreAction(option_strings=['--include_title'], dest='include_title', nargs=None, const=None, default=True, type=<function str2bool at 0x7f89837c1a60>, choices=None, help=None, metavar=None)

In [3]:
args,_ = parser.parse_known_args()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('../models/klue_bert_tokenizer')

In [5]:
bm25 = BM25(args, tokenizer)
bm25.exec_embedding()

tokenize: 100%|█████████████████████| 1346460/1346460 [05:13<00:00, 4294.48it/s]


In [6]:
test_data = load_jsonl('./data/train.jsonl')

51237it [00:00, 150930.57it/s]


In [7]:
test_data = load_jsonl('./data/dev.jsonl')

4909it [00:00, 130616.35it/s]


In [8]:
result = []
for i in tqdm(test_data):
    output = bm25.retrieve(i['question'],100)
    output = [i[0] for i in output]
    result.append(output)

100%|████████████████████████████████████| 4909/4909 [13:39:41<00:00, 10.02s/it]


In [17]:
# hit@1, 5, 20, 100
scores = []
for k in [1,5,20,100]:
    score_k = []
    for i,j in tqdm(zip(test_data,result)):
        score_k.append(i['positive_ctxs_ids'][0] in j[:k])
    scores.append(score_k)

4909it [00:00, 391717.34it/s]
4909it [00:00, 357406.63it/s]
4909it [00:00, 277016.94it/s]
4909it [00:00, 171128.50it/s]


In [18]:
for i in scores:
    print(sum(i)/len(i))

0.567936443267468
0.7751069464249338
0.8653493583214504
0.927072723568955


In [10]:
for i in tqdm(test_data):
    output = bm25.retrieve_without(i['question'],i['answer'],100,1)
    i['hard_negative_ctxs_ids']=output

  0%|                                       | 1/4909 [00:13<17:48:51, 13.07s/it]


KeyboardInterrupt: 